# Creating a streamlit app

## Requirements

Using in my case streamlit version 0.65.2

In [9]:
import streamlit as st
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk import FreqDist
from lexical_diversity import lex_div as ld

pd.options.display.max_columns = None

In [1]:
!pip3 install streamlit

    100% |████████████████████████████████| 7.2MB 166kB/s ta 0:00:011
    100% |████████████████████████████████| 102kB 2.3MB/s a 0:00:011
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/94/d47b0fd5988e6b7059de05720a646a2930920fff247a826f61674d436ba4/tzlocal-2.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 10.5MB 125kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/45/1e/0c169c6a5381e241ba7404532c16a21d86ab872c9bed8bdcd4c423954103/requests-2.24.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/e7/4b2bdddb99f5f631d8c1de259897c2b7d65dcfcc1e0a6fd17a7f62923500/numpy-1.19.1-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 2.2MB 445kB/s ta 0:00:01
    100% |████████████████████████████████| 737kB 1.2MB/s ta 0:00:01
    100% |██████

  Using cached https://files.pythonhosted.org/packages/d1/29/605c2cc68a9992d18dada28206eeada56ea4bd07a239669da41674648b6f/ptyprocess-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/59/7c/e39aca596badaf1b78e8f547c807b04dae603a433d3e7a7e04d67f2ef3e5/wcwidth-0.2.5-py2.py3-none-any.whl
    100% |████████████████████████████████| 102kB 1.3MB/s a 0:00:011
  Using cached https://files.pythonhosted.org/packages/ff/96/1d9a2c23990aea8f8e0b5c3b6627d03196a73771a17a2d9860bbe9823ab6/terminado-0.8.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/0e/554a265ffdc56e1494ef08e18f765b0cdec78797f510c58c45cf37abb4f4/prometheus_client-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/6c/05a569e9f703d18aacb89b7ad6075b404e8a4afde2c26b73ca77bb644b14/nbconvert-5.6.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 409kB 1.3MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4c/ea

In [2]:
!streamlit --version

Streamlit, version 0.65.2


# Index:
### 1. Functions for the app:

- [`Get news_features_function()`](#Get-news_features-function:-Necessary-to-extract-features-from-the-headline-and-new's-content)
- [`Get_predictions_function()`](#Get_predictions_function)

### [2. Extract news with url](#2.-Extract-news-with-url)

### [3. Streamlit configuration](#3.-Streamlit-config)

### [4. Full script](#4.-Full-script)

### `get_news_features function()`: Necessary to extract features from the headline and new's content
Also we will need `getn_syllables()`if we want that `get_news_features function()` function works. 

In [ ]:
# @st.cache(show_spinner = False)
def get_nsyllables(text):
    from syltippy import syllabize
    
    text = text.replace(r"*URL*", "url")
    text = re.sub(r'\d+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^ \nA-Za-z0-9ÁÉÍÓÚÑáéíóúñ/]+', '', text)
    
    n_syllables = len(syllabize(text)[0])
    
    return n_syllables

In [10]:
# @st.cache(show_spinner = False)
%%time

import pandas as pd
import nltk
import spacy
import re
from nltk import FreqDist
from lexical_diversity import lex_div as ld

def get_news_features(headline, text):
    
    nlp = spacy.load('es_core_news_lg')

    ## headline ##
    headline = re.sub(r"http\S+", "", headline)
    headline = re.sub(r"http", "", headline)
    headline = re.sub(r"@\S+", "", headline)
    headline = re.sub("\n", " ", headline)
    headline = re.sub(r"(?<!\n)\n(?!\n)", " ", headline)
    headline = headline.replace(r"*NUMBER*", "número")
    headline = headline.replace(r"*PHONE*", "número")
    headline = headline.replace(r"*EMAIL*", "email")
    headline = headline.replace(r"*URL*", "url")
    headline_new = headline.lower()
    doc_h = nlp(headline_new)

    list_tokens_h = []
    list_tags_h = []

    for sentence_h in doc_h.sents:
        for token in sentence_h:
            list_tokens_h.append(token.text)

    fdist_h = FreqDist(list_tokens_h)
    syllables_h = get_nsyllables(headline)
    words_h = len(list_tokens_h)

    # headline complexity features
    avg_word_size_h = round(sum(len(word) for word in list_tokens_h) / words_h, 2)
    avg_syllables_word_h = round(syllables_h / words_h, 2)
    unique_words_h = round((len(fdist_h.hapaxes()) / words_h) * 100, 2)
    mltd_h = round(ld.mtld(list_tokens_h), 2)
    ttr_h = round(ld.ttr(list_tokens_h) * 100, 2)

    ## text content##     
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)
    text = re.sub("\n", " ", text)
    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")

    # to later calculate upper case letters ratio
    alph = list(filter(str.isalpha, text))
    text_lower = text.lower()
    doc = nlp(text_lower)

    list_tokens = []
    list_pos = []
    list_tag = []
    list_entities = []
    sents = 0

    for entity in doc.ents:
        list_entities.append(entity.label_)

    for sentence in doc.sents:
        sents += 1
        for token in sentence:
            list_tokens.append(token.text)
            list_pos.append(token.pos_)
            list_tag.append(token.tag_)

    # Calculate entities, pos, tag, freq, syllables, words and quotes
    entities = len(list_entities)
    n_pos = nltk.Counter(list_pos)
    n_tag = nltk.Counter(list_tag)
    fdist = FreqDist(list_tokens)
    syllables = get_nsyllables(text)
    words = len(list_tokens)
    quotes = n_tag['PUNCT__PunctType=Quot']

    # complexity features
    avg_word_sentence = round(words / sents, 2)
    avg_word_size = round(sum(len(word) for word in list_tokens) / words, 2)
    avg_syllables_word = round(syllables / words, 2)
    unique_words = round((len(fdist.hapaxes()) / words) * 100, 2)
    ttr = round(ld.ttr(list_tokens) * 100, 2)

    # readability spanish test
    huerta_score = round(206.84 - (60 * avg_syllables_word) - (1.02 * avg_word_sentence), 2)
    szigriszt_score = round(206.835 - ((62.3 * syllables) / words) - (words / sents), 2)

    # stylometric features
    mltd = round(ld.mtld(list_tokens), 2)
    upper_case_ratio = round(sum(map(str.isupper, alph)) / len(alph) * 100, 2)
    entity_ratio = round((entities / words) * 100, 2)
    quotes_ratio = round((quotes / words) * 100, 2)
    propn_ratio = round((n_pos['PROPN'] / words) * 100 , 2)
    noun_ratio = round((n_pos['NOUN'] / words) * 100, 2) 
    pron_ratio = round((n_pos['PRON'] / words) * 100, 2)
    adp_ratio = round((n_pos['ADP'] / words) * 100, 2)
    det_ratio = round((n_pos['DET'] / words) * 100, 2)
    punct_ratio = round((n_pos['PUNCT'] / words) * 100, 2)
    verb_ratio = round((n_pos['VERB'] / words) * 100, 2)
    adv_ratio = round((n_pos['ADV'] / words) * 100, 2)
    sym_ratio = round((n_tag['SYM'] / words) * 100, 2)

    # create df_features
    df_features = pd.DataFrame({'words_h': words_h, 'word_size_h': [avg_word_size_h],'avg_syllables_word_h': [avg_syllables_word_h],
                                'unique_words_h': [unique_words_h], 'ttr_h': ttr_h, 'mltd_h': [mltd_h], 'sents': sents, 'words': words,
                                'avg_words_sent': [avg_word_sentence], 'avg_word_size': [avg_word_size], 
                                'avg_syllables_word': avg_syllables_word, 'unique_words': [unique_words], 
                                'ttr': [ttr], 'huerta_score': [huerta_score], 'szigriszt_score': [szigriszt_score],
                                'mltd': [mltd], 'upper_case_ratio': [upper_case_ratio], 'entity_ratio': [entity_ratio],
                                'quotes': quotes, 'quotes_ratio': [quotes_ratio], 'propn_ratio': [propn_ratio], 
                                'noun_ratio': [noun_ratio], 'pron_ratio': [pron_ratio], 'adp_ratio': [adp_ratio],
                                'det_ratio': [det_ratio], 'punct_ratio': [punct_ratio], 'verb_ratio': [verb_ratio],
                                'adv_ratio': [adv_ratio], 'sym_ratio': [sym_ratio]})
    
    return df_features

### `Get_predictions function()`

In [11]:
# @st.cache(show_spinner = False)
def get_predictions(pickle_file, df_features):
    with st.spinner("Detectando..."):
        import pickle
        model = pickle.load(open(pickle_file, 'rb'))

        # prediction
        prediction = (model.predict(df_features)[0])
        prob_fake = (model.predict_proba(df_features)[0][0])*100
        prob_real = (model.predict_proba(df_features)[0][1])*100
    
    return prediction, prob_fake, prob_real

## 2. Extract news with url

### ¿How to extract the headline and New's content directly with and url?

This is possible with __[Newspaper3k: Article scraping & curation](https://github.com/codelucas/newspaper)__ library for python that extracts the headline and the new's content from any newspaper

In [37]:
from newspaper import Article

url = input('Insert url: ')

article = Article(url, language = 'es')
article.download()
article.parse()

df_features = get_news_features(article.title, article.text)

Insert url: https://edition.cnn.com/2020/09/05/politics/michael-cohen-book-trump-white-house/index.html


In [12]:
import tldextract

url = 'https://www.eldiario.es/ballenablanca/biodiversidad/estudio-publicado-nature-advierte-impacto-biodiversidad-mineria-producir-renovables_1_6201934.html'

ext = tldextract.extract(url)
ext.domain

ExtractResult(subdomain='www', domain='eldiario', suffix='es')

In [39]:
def extract_news(url):
    with st.spinner("Detectando..."):
        from newspaper import Article
        import tldextract
        article = Article(url)
        article.download()
        article.parse()
        ext = tldextract.extract(url)
        newspaper = ext.domain
        
        headline = article.title
        text = article.text
        image_url = article.top_image
        
        if article.author == []:
            author = []
        else:
            author = article.author
            
        return headline, text, author, newspaper, image_url   

## 3. Streamlit config

In [13]:
# page configuration
page_title = 'Fake News Detector'
layout = 'wide'
initial_sidebar_state = 'expanded'

# display title and description
st.title("Fake news predictor based in spanish language news")
"""
Master Thesis in Data Science - KSchool

This app is powered with a Machine Learning Algorithm to detect which news in spanish language are fake or legitimate
Insert the new's headline and new's content text, and lets detect!
"""

url = st.text_input("Inserta el enlace de la noticia: ")

headline_text = st.text_input("Pega el titular de la noticia: ")
news_text = st.text_input("Pega el texto de la noticia: ")


if prob_fake >= 65:
    st.write('Esta noticia es falsa.\nCon una probabilidad del %.0f%%' % prob_fake)

elif (65 >= prob_fake >= 35):
    st.write('Esta noticia es engañosa o una opinión. \nTiene una probabilidad de %.0f%% de ser verdadera' % prob_real)
    
else:
    st.write('Esta noticia es verdadera.\nCon una probabilidad del %.0f%%' % prob_real)

NameError: name 'st' is not defined

In [25]:
import lexical_diversity

print(lexical_diversity.__version__)

AttributeError: module 'lexical_diversity' has no attribute '__version__'

## 4. Full script

In [5]:
%%writefile fake_news_detector_app.py

import streamlit as st
import pandas as pd
import numpy as np
import nltk
import re
import spacy
#need to switch to es_core_news_md pacy model. es_core_news_lg too large for Heroku.
import es_core_news_md
from nltk import FreqDist
from lexical_diversity import lex_div as ld

#### extract news with url ####

@st.cache(show_spinner = False)
def extract_news(url):
    with st.spinner("Detectando..."):
        from newspaper import Article
        import tldextract
        article = Article(url)
        article.download()
        article.parse()
        ext = tldextract.extract(url)
        newspaper = ext.domain
        image_url = article.top_image
        
        headline = article.title
        text = article.text
        
        if article.authors == []:
            author = []
        else:
            author = article.authors
            
        return headline, text, author, newspaper, image_url

@st.cache(show_spinner = False)
def get_nsyllables(text):
    from syltippy import syllabize
    
    text = text.replace(r"*URL*", "url")
    text = re.sub(r'\d+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^ \nA-Za-z0-9ÁÉÍÓÚÑáéíóúñ/]+', '', text)
    
    n_syllables = len(syllabize(text)[0])
    
    return n_syllables


#### text features ####

@st.cache(show_spinner = False)
def get_news_features(headline, text):
    
    nlp = es_core_news_md.load()

    ## headline ##
    headline = re.sub(r"http\S+", "", headline)
    headline = re.sub(r"http", "", headline)
    headline = re.sub(r"@\S+", "", headline)
    headline = re.sub("\n", " ", headline)
    headline = re.sub(r"(?<!\n)\n(?!\n)", " ", headline)
    headline = headline.replace(r"*NUMBER*", "número")
    headline = headline.replace(r"*PHONE*", "número")
    headline = headline.replace(r"*EMAIL*", "email")
    headline = headline.replace(r"*URL*", "url")
    headline_new = headline.lower()
    doc_h = nlp(headline_new)

    list_tokens_h = []
    list_tags_h = []

    for sentence_h in doc_h.sents:
        for token in sentence_h:
            list_tokens_h.append(token.text)

    fdist_h = FreqDist(list_tokens_h)
    syllables_h = get_nsyllables(headline)
    words_h = len(list_tokens_h)

    # headline complexity features
    avg_word_size_h = round(sum(len(word) for word in list_tokens_h) / words_h, 2)
    avg_syllables_word_h = round(syllables_h / words_h, 2)
    unique_words_h = round((len(fdist_h.hapaxes()) / words_h) * 100, 2)
    mltd_h = round(ld.mtld(list_tokens_h), 2)
    ttr_h = round(ld.ttr(list_tokens_h) * 100, 2)

    ## text content##     
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)
    text = re.sub("\n", " ", text)
    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")

    # to later calculate upper case letters ratio
    alph = list(filter(str.isalpha, text))
    text_lower = text.lower()
    doc = nlp(text_lower)

    list_tokens = []
    list_pos = []
    list_tag = []
    list_entities = []
    sents = 0

    for entity in doc.ents:
        list_entities.append(entity.label_)

    for sentence in doc.sents:
        sents += 1
        for token in sentence:
            list_tokens.append(token.text)
            list_pos.append(token.pos_)
            list_tag.append(token.tag_)

    # Calculate entities, pos, tag, freq, syllables, words and quotes
    entities = len(list_entities)
    n_pos = nltk.Counter(list_pos)
    n_tag = nltk.Counter(list_tag)
    fdist = FreqDist(list_tokens)
    syllables = get_nsyllables(text)
    words = len(list_tokens)
    quotes = n_tag['PUNCT__PunctType=Quot']

    # complexity features
    avg_word_sentence = round(words / sents, 2)
    avg_word_size = round(sum(len(word) for word in list_tokens) / words, 2)
    avg_syllables_word = round(syllables / words, 2)
    unique_words = round((len(fdist.hapaxes()) / words) * 100, 2)
    ttr = round(ld.ttr(list_tokens) * 100, 2)

    # readability spanish test
    huerta_score = round(206.84 - (60 * avg_syllables_word) - (1.02 * avg_word_sentence), 2)
    szigriszt_score = round(206.835 - ((62.3 * syllables) / words) - (words / sents), 2)

    # stylometric features
    mltd = round(ld.mtld(list_tokens), 2)
    upper_case_ratio = round(sum(map(str.isupper, alph)) / len(alph) * 100, 2)
    entity_ratio = round((entities / words) * 100, 2)
    quotes_ratio = round((quotes / words) * 100, 2)
    propn_ratio = round((n_pos['PROPN'] / words) * 100 , 2)
    noun_ratio = round((n_pos['NOUN'] / words) * 100, 2) 
    pron_ratio = round((n_pos['PRON'] / words) * 100, 2)
    adp_ratio = round((n_pos['ADP'] / words) * 100, 2)
    det_ratio = round((n_pos['DET'] / words) * 100, 2)
    punct_ratio = round((n_pos['PUNCT'] / words) * 100, 2)
    verb_ratio = round((n_pos['VERB'] / words) * 100, 2)
    adv_ratio = round((n_pos['ADV'] / words) * 100, 2)
    sym_ratio = round((n_tag['SYM'] / words) * 100, 2)

    # create df_features
    df_features = pd.DataFrame({'words_h': words_h, 'word_size_h': [avg_word_size_h],'avg_syllables_word_h': [avg_syllables_word_h],
                                'unique_words_h': [unique_words_h], 'ttr_h': ttr_h, 'mltd_h': [mltd_h], 'sents': sents, 'words': words,
                                'avg_words_sent': [avg_word_sentence], 'avg_word_size': [avg_word_size], 
                                'avg_syllables_word': avg_syllables_word, 'unique_words': [unique_words], 
                                'ttr': [ttr], 'huerta_score': [huerta_score], 'szigriszt_score': [szigriszt_score],
                                'mltd': [mltd], 'upper_case_ratio': [upper_case_ratio], 'entity_ratio': [entity_ratio],
                                'quotes': quotes, 'quotes_ratio': [quotes_ratio], 'propn_ratio': [propn_ratio], 
                                'noun_ratio': [noun_ratio], 'pron_ratio': [pron_ratio], 'adp_ratio': [adp_ratio],
                                'det_ratio': [det_ratio], 'punct_ratio': [punct_ratio], 'verb_ratio': [verb_ratio],
                                'adv_ratio': [adv_ratio], 'sym_ratio': [sym_ratio]})
    
    return df_features  


#### predictions ####

@st.cache(show_spinner = False)
def get_predictions(pickle_file, df_features):
    with st.spinner("Detectando..."):
        import pickle
        model = pickle.load(open(pickle_file, 'rb'))
        
        numeric_features = ['words_h', 'word_size_h', 'avg_syllables_word_h', 'unique_words_h', 'ttr_h', 'mltd_h', 'sents',
                            'words', 'avg_words_sent', 'avg_word_size', 'avg_syllables_word', 'unique_words', 'ttr', 'mltd', 
                            'huerta_score', 'szigriszt_score','upper_case_ratio', 'entity_ratio', 'quotes', 'quotes_ratio',
                            'propn_ratio', 'noun_ratio', 'adp_ratio', 'det_ratio', 'punct_ratio', 'pron_ratio', 'verb_ratio', 
                            'adv_ratio', 'sym_ratio']

        # prediction
        prediction = (model.predict(df_features[numeric_features])[0])
        prob_fake = (model.predict_proba(df_features[numeric_features])[0][0])*100
        prob_real = (model.predict_proba(df_features[numeric_features])[0][1])*100
    
    return prob_fake, prob_real

#### streamlit configuration ####

# load pickle file 
pickle_file = './fake_news_predictorv3.pkl'

# page configuration
page_title = 'Fake News Detector'
layout = 'wide'
initial_sidebar_state = 'expanded'

# display title and description
st.title("Detector de")
st.image('./FAKE_NEWS_title.png', use_column_width = True, width = None, output_format = 'auto')

# text input for headline and new's content
url = st.text_input("Pega el enlace de la noticia.")

## run functions##
if (url != ""):
    
    headline, text, author, newspaper, image_url = extract_news(url)
    df_features = get_news_features(headline, text)
    prob_fake, prob_real = get_predictions(pickle_file, df_features)


    if prob_fake >= 65:
        st.error('¡¡Esta noticia es **FALSA**!! :heavy_multiplication_x: \nCon una probabilidad del %d%%.' % int(prob_fake))

    elif (65 >= prob_fake >= 35):
        st.warning('¡Esta noticia es **ENGAÑOSA**! :exclamation: \nTiene una probabilidad de %d%% de ser verdadera.' % int(prob_real))

    else:
        st.success('Esta noticia es **VERDADERA** :white_check_mark: \nCon una probabilidad del %d%%.' % int(prob_real))
    
    st.title(headline)
    st.write(newspaper)
    st.image(image_url, use_column_width = True, width = None)
    if author == []:
        st.write('Autor no encontrado')
    else:
        st.write('Autor: ', author[0])
    st.write('Noticia: ', text)

Overwriting fake_news_detector_app.py


In [6]:
!streamlit run fake_news_detector_app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://192.168.43.219:8501
  External URL: http://176.87.16.101:8501

image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
image/jpeg
^C
  Stopping...
